### Proxy Pattern

* Proxy Pattern lets you provide a substitute or placeholder for another object. The proxy controls access to the original object. Must ensure that the proxy implements the same interface as the original object

In [4]:
from abc import ABC, abstractmethod

class YouTubeLibrary(ABC):
    @abstractmethod
    def listVideos(self):
        pass

    @abstractmethod
    def getVideos(self):
        pass

    @abstractmethod
    def downloadVideos(self):
        pass


class ThirdPartyYouTubeLibrary(YouTubeLibrary):
    def __init__(self):
        self.videos = {1: "sleeping.avi", 2: "Hacking.avi", 3:"Soccer.avi"}

    def listVideos(self):
        return self.videos.values()
    
    def getVideos(self,id):
        if id in self.videos:
            return self.videos[id]
        
    def downloadVideos(self,id):
        return self.getVideos(id)
    
class CachedYouTubeLibrary(YouTubeLibrary):
    def __init__(self,library: ThirdPartyYouTubeLibrary):
        self.library = library
        self.cache = []

    def listVideos(self):
        if len(self.cache) == 0:
            videos = self.library.listVideos()
            self.cache = videos
            return videos
        else:
            return self.cache
        

    def getVideos(self,id):
        if len(self.cache) == 0:
            video = self.library.getVideos(id)
            self.cache = [video]
            return video
        
        else:
            return self.cache[0]
        
    def downloadVideos(self,id):
        return self.getVideos(id)
    

thirdPartyYouTubeLibrary = ThirdPartyYouTubeLibrary()

cachedYouTubeLibrary = CachedYouTubeLibrary(thirdPartyYouTubeLibrary)

videos = cachedYouTubeLibrary.listVideos()


print(videos)
        


dict_values(['sleeping.avi', 'Hacking.avi', 'Soccer.avi'])


### Behavorial Pattern

* This pattern is more involved on the interaction between two or more objects

* Observer Pattern
* Mediattor Pattern
* Command Pattern

* Observer Pattern

* producer ->Event Notifier  -> one or more consumers

* consumers subscribe to a particular event on the Event notifier
* producer produces the event and sends to the event notifier
* Event Notifier now send the event to the consumers that subscribed to the event

In [9]:
from abc import ABC, abstractmethod

### inteface for our consumers or listeners
class EventListener(ABC):
    @abstractmethod
    def update(self,message):
        pass

######## Event Notifier
class EventNotifier:
    def __init__(self):
        self.listener = {}
    # {"log":[loglistner, loglistener2, loglistener3], "email":[emaillistener]}
    def subscribe(self,eventType, listener: EventListener):
        if eventType in self.listener:
            self.listener[eventType].append(listener)
        else:
            self.listener[eventType] = [listener]

    def unsubscribe(self,eventType, listener: EventListener):
        if eventType in self.listener:
            self.listener[eventType].remove(listener)

        raise Exception("Unable to Unsubscribe")


    def notify(self,eventType, data):
        for listener in self.listener[eventType]:
            listener.update(data)


######## producers

class Producer:
    def __init__(self, eventNotifier: EventNotifier):
        self.eventNotifier = eventNotifier

    def produceEvent(self,eventType):
        message = "testing observer pattern"
        self.eventNotifier.notify(eventType,message)

######## consumers
class LoggingListener(EventListener):
    def update(self, message):
        print(f'Logging message: {message}')


class EmailAlertsListener(EventListener):
    def update(self, message):
        print(f'Sending email with body: {message}')

# create instance of event notifier
eventNotifier = EventNotifier()

# create consumers
logListener = LoggingListener()
emailAlertsListener = EmailAlertsListener()

# consumers subscribe to an event
eventNotifier.subscribe("log", logListener)
eventNotifier.subscribe("email", emailAlertsListener)

# producer prodices the event
producer = Producer(eventNotifier)

producer.produceEvent("email")
#producer.produceEvent("email")








Sending email with body: testing observer pattern


### Mediator Pattern

* A pattern the restricts direct communication between the objects and forces them to collaborate only via the mediator object



In [5]:
from abc import ABC, abstractmethod

class Dialog:
    def notify(self, sender, receiver, message):
        sender.send(receiver, message)


class Component(ABC):
    @abstractmethod
    def send(self, receiver, message):
        pass

    @abstractmethod
    def receive(self, sender, message):
        pass


class Button(Component):
    def send(self, receiver, message):
        receiver.receive(self,message)

    def receive(self, sender, message):
        print(f"received message {message} from sender: {sender}")


class TextBox(Component):

    def send(self, receiver, message):
        receiver.receive(self, message)

    def receive(self, sender, message):
        print(f"received message {message} from sender: {sender}")


dialog = Dialog()

button = Button()
textbox = TextBox()


dialog.notify(button, textbox, "the was clicked. Enable yourself")




received message the was clicked. Enable yourself from sender: <__main__.Button object at 0x104602170>


### Class Activity

You are building an online store. Where customers want to receive different types of notification sms, email and mobile notifications.
Create classes to simulate the observer pattern for this problem.

* create a producer class that produces each of the notification types above
* create an Event notifier class for consumers to subscribe to notifications
* create aleast one consumer class that will receieve any of the notification types above

In [7]:
from abc import ABC, abstractmethod

class Consumer(ABC):
    @abstractmethod
    def update(self,message):
        pass


class NotificationService:
    def __init__(self):
        self.container = {}

    def subscribe(self,consumer:Consumer, event):
        if event in self.container:
            self.container[event].append(consumer)

        else:
            self.container[event] = [consumer]

    def unsubscribe(self,consumer:Consumer, event):
        if event in self.container:
            self.container[event].remove(consumer)

    def notify(self,event):
        for consumer in self.container[event]:
            consumer.update(event)

class SmsConsumer(Consumer):
    def update(self,message):
        print("received sms notification")

class EmailConsumer(Consumer):
    def update(self,message):
        print("Email nofication received ")


class Producer:
    def __init__(self, service: NotificationService):
        self.service = service
    def produceEvent(self, event):
        self.service.notify(event)


notificationService = NotificationService()

smsConsumer = SmsConsumer()
emailConsumer = EmailConsumer()

notificationService.subscribe(smsConsumer,"sms")
notificationService.subscribe(emailConsumer, "email")

producer = Producer(notificationService)

producer.produceEvent("email")


Email nofication received 


### Command Pattern

* The request from client applications should come in as a stand alone object. Therefore, all transformations that should be performed on request ahould be done in the object



In [10]:
from typing import List
from abc import ABC, abstractmethod


class Command(ABC):
    @abstractmethod
    def CreateOrder(self,name,orderItems):
        pass


class OrderCommand(Command):
    def __init__(self,name, orderItems):
        self.name = name
        self.orderItems = orderItems

    def CreateOrder(self):
        totalPrice = 0.0
        for orderItem in self.orderItems:
            totalPrice += (orderItem.quantity * orderItem.unitPrice)

        return Order(self.name, self.orderItems, totalPrice)


class OrderItem:
    def __init__(self,productName, quantity,unitPrice):
        self.productName = productName
        self.quantity = quantity
        self.unitPrice = unitPrice


class Order:
    def __init__(self, name, orderItems: List[OrderItem], totalPrice: float):
        self.name = name
        self.orderItems = orderItems
        self.totalPrice = totalPrice

class OrderService:
    def __init__(self, command: Command):
        self.command = command

    def CreateOrder(self):
        return self.command.CreateOrder()
        
    

orderItems = [OrderItem("Coke",2,10), OrderItem("Milk",1,15)]

orderCommand = OrderCommand('fred', orderItems)
orderService = OrderService(orderCommand)

order = orderService.CreateOrder()

print(order.name)
for orderItem in orderItems:
    print(orderItem.productName)
    print(orderItem.unitPrice)
    print(orderItem.quantity)
print(order.totalPrice)


fred
Coke
10
2
Milk
15
1
35.0
